In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/캡스톤/Final_Data.csv')
df.head()

,Movie,Summary,Genre,Emotion,4Emotion,PN,CS,Instrument
0,천하장사마돈나,"Two people sit on a bench, one taking pictures...",드라마,-3,D,0,1,키보드
1,천하장사마돈나,"During an English class, a camera captures the...",판타지,3,A,1,1,키보드
2,천하장사마돈나,A person named Dong-gu is sitting at a desk an...,판타지,4,A,1,1,음율타악기
3,천하장사마돈나,A teacher and student embrace after the studen...,판타지,2,B,1,0,키보드
4,천하장사마돈나,A student named Dong-gu is excited to see his ...,로맨스,1,B,1,0,키보드


In [ ]:
df.Genre.unique()

array(['드라마', '판타지', '로맨스', '일상', '코미디', '스릴러', '범죄', '액션'], dtype=object)

In [ ]:
df.loc[df['Genre'] == '판타지', 'Genre'] = 0
df.loc[df['Genre'] == '로맨스', 'Genre'] = 1
df.loc[df['Genre'] == '일상', 'Genre'] = 2
df.loc[df['Genre'] == '코미디', 'Genre'] = 3
df.loc[df['Genre'] == '스릴러', 'Genre'] = 4
df.loc[df['Genre'] == '액션', 'Genre'] = 5
df.loc[df['Genre'] == '범죄', 'Genre'] = 6
df.loc[df['Genre'] == '드라마', 'Genre'] = 7

In [ ]:
df = df.astype({'Genre':'int64'})

In [ ]:
df_data_shuffled = df.sample(frac=1).reset_index(drop=True)

In [ ]:
#train data & test data 로드
train = df_data_shuffled[:2500]
test = df_data_shuffled[2500:]

In [ ]:
train.to_csv('/content/drive/MyDrive/캡스톤/train.csv')

In [ ]:
test.to_csv('/content/drive/MyDrive/캡스톤/test.csv')

In [ ]:
# CLS, SEP 붙이기 (문장의 시작, 끝)
sentences = ["[CLS] " + str(s) + " [SEP]" for s in train.Summary]

In [ ]:
labels=train['Genre'].values

In [ ]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
result = tokenizer.tokenize('hello')
print(result)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

['hell', '##o']


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]

In [ ]:
print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후

[CLS] Six coal-covered miners sit, unable to be distinguished from one another. They eat and drink from their lunch pails, some chewing bread and others potatoes. One miner, Duk-soo, stares blankly as he chews his bread. [SEP]
['[CLS]', 'Six', 'coal', '-', 'covered', 'mine', '##rs', 'sit', ',', 'unable', 'to', 'be', 'distinguished', 'from', 'one', 'another', '.', 'They', 'eat', 'and', 'drink', 'from', 'their', 'lu', '##nch', 'pai', '##ls', ',', 'some', 'che', '##wing', 'br', '##ead', 'and', 'others', 'pot', '##ato', '##es', '.', 'One', 'mine', '##r', ',', 'Du', '##k', '-', 'soo', ',', 'stare', '##s', 'bl', '##ank', '##ly', 'as', 'he', 'che', '##ws', 'his', 'br', '##ead', '.', '[SEP]']


In [ ]:
MAX_LEN = 128 #최대 시퀀스 길이 설정
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels,
                                                                                    random_state=2000,
                                                                                    test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks,
                                                       input_ids,
                                                       random_state=2000,
                                                       test_size=0.1)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [ ]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
# [CLS] + 문장 + [SEP]
sentences2 = ["[CLS] " + str(sentence) + " [SEP]" for sentence in  test.Summary]

In [ ]:
# 라벨 데이터
labels = test['Genre'].values

# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences2]

# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# 배치 사이즈 설정 및 데이터 설정
batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=8)
model.cuda()

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):

    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)


# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))

    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # 그래디언트 계산 안함
        with torch.no_grad():
            # Forward 수행
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 1.94
  Training epcoh took: 0:00:49

Running Validation...
  Accuracy: 0.32
  Validation took: 0:00:02

======== Epoch 2 / 4 ========
Training...

  Average training loss: 1.62
  Training epcoh took: 0:00:49

Running Validation...
  Accuracy: 0.41
  Validation took: 0:00:02

======== Epoch 3 / 4 ========
Training...

  Average training loss: 1.41
  Training epcoh took: 0:00:48

Running Validation...
  Accuracy: 0.43
  Validation took: 0:00:02

======== Epoch 4 / 4 ========
Training...

  Average training loss: 1.28
  Training epcoh took: 0:00:48

Running Validation...
  Accuracy: 0.42
  Validation took: 0:00:02

Training complete!


In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)

    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.46
Test took: 0:00:03


In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return np.argmax(logits)

In [ ]:
a='청명 draws his sword and warns those who oppose him to leave, citing a recent change in his attitude. When challenged by scoffers, he signals his elite guards to step forward. Ignoring further dialogue, he issues a final warning before commanding his men to attack. Surprising everyone, including intimidated merchants, he effortlessly defeats a group of skilled warriors, leaving them incapacitated on the ground. With a confident look, he turns his attention to the remaining opponent, Gongmunyeon.'

In [ ]:
test_sentences([a])

5

In [ ]:
test.reset_index(drop=True, inplace=True)

In [ ]:
test.Summary[0]

'Children go to the auditorium to prepare for a festival at a school where it is raining. However, they see blood stains at the entrance. The volunteer has taken a large amount of pills and looks terrible with blood all over their body.'

In [ ]:
test['Pred_PN']=1

<ipython-input-63-0aba3fa18922>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Pred_PN']=1


In [ ]:
for i in range(len(test)):
    logits =test_sentences([test.Summary[i]])
    test.Pred_PN[i]=np.argmax(logits)

<ipython-input-64-0198af0a4059>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Pred_PN[i]=np.argmax(logits)


In [ ]:
test

,Movie,Summary,Genre,Emotion,4Emotion,PN,CS,Instrument,Pred_PN
0,피의 고사,Children go to the auditorium to prepare for a...,4,-2,C,0,0,피아노,4
1,jsa,The input includes dialogue and analysis resul...,7,-1,C,0,0,키보드,4
2,7급공무원,A conversation between three people is interru...,3,2,B,1,0,관악기,1
3,탐정,A mid-30s man from Taiwan sits in front of a c...,7,-1,C,0,0,음율타악기,7
4,악마를 보았다,A man named Daehoon is standing and looking at...,7,-2,C,0,0,피아노,7
...,...,...,...,...,...,...,...,...,...
494,신라의달밤,Chul-gyu takes a nap and is suddenly awoken by...,5,-3,D,0,1,타악기,4
495,GP506,A military officer and medic attempt to revive...,7,-2,C,0,0,현악기,7
496,식객,A man named Man-sik challenges Bong-ju to a fi...,7,-1,C,0,0,현악기,7
497,퍼펙트게임,A person named Dongwon throws objects until th...,7,-3,D,0,1,피아노,2


In [ ]:
sum(test.Genre==test.Pred_PN)/499

0.45490981963927857

In [ ]:
df.loc[df['Genre'] == '판타지', 'Genre'] = 0
df.loc[df['Genre'] == '로맨스', 'Genre'] = 1
df.loc[df['Genre'] == '일상', 'Genre'] = 2
df.loc[df['Genre'] == '코미디', 'Genre'] = 3
df.loc[df['Genre'] == '스릴러', 'Genre'] = 4
df.loc[df['Genre'] == '액션', 'Genre'] = 5
df.loc[df['Genre'] == '범죄', 'Genre'] = 6
df.loc[df['Genre'] == '드라마', 'Genre'] = 7

In [ ]:
test.loc[test['Pred_PN'] == 0 ,'Pred_PN'] = 1
test.loc[test['Pred_PN'] == 1,'Pred_PN'] = 1
test.loc[test['Pred_PN'] ==2 ,'Pred_PN'] = 1
test.loc[test['Pred_PN'] == 3,'Pred_PN'] = 1
test.loc[test['Pred_PN'] ==4 ,'Pred_PN'] = 0
test.loc[test['Pred_PN'] ==5 ,'Pred_PN'] = 0
test.loc[test['Pred_PN'] ==6 ,'Pred_PN'] = 0
test.loc[test['Pred_PN'] ==7 ,'Pred_PN'] = 0

In [ ]:
sum(test.PN==test.Pred_PN)/499

0.7695390781563126

In [ ]:
test.to_csv('/content/drive/MyDrive/캡스톤/sb.csv')